In [2]:
import pandas as pd

# Define file paths
test_file_path = '~/mol/data/handwriting/test.csv'
train_file_path = '~/mol/data/handwriting/train.csv'

# Load the CSV files into pandas DataFrames
test_df = pd.read_csv(test_file_path)
train_df = pd.read_csv(train_file_path)

# Display the first few rows of each DataFrame

print("\nTrain DataFrame:")
train_df.head()

Test DataFrame:

Train DataFrame:


,writer,page_id,language,same_text,tortuosityHist10[0],tortuosityHist10[1],tortuosityHist10[2],tortuosityHist10[3],tortuosityHist10[4],tortuosityHist10[5],...,directions_hist1a2a3a4a5a6a7a8a9a10_220[210],directions_hist1a2a3a4a5a6a7a8a9a10_220[211],directions_hist1a2a3a4a5a6a7a8a9a10_220[212],directions_hist1a2a3a4a5a6a7a8a9a10_220[213],directions_hist1a2a3a4a5a6a7a8a9a10_220[214],directions_hist1a2a3a4a5a6a7a8a9a10_220[215],directions_hist1a2a3a4a5a6a7a8a9a10_220[216],directions_hist1a2a3a4a5a6a7a8a9a10_220[217],directions_hist1a2a3a4a5a6a7a8a9a10_220[218],directions_hist1a2a3a4a5a6a7a8a9a10_220[219]
0,1,1,Arabic,0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.002457,0.002633,0.002698,0.002929,0.003014,0.003225,0.003572,0.003957,0.004232,0.004804
1,1,2,Arabic,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.002498,0.002641,0.002836,0.002999,0.003246,0.003456,0.003709,0.003994,0.004308,0.004706
2,1,3,English,0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.002090,0.002440,0.002831,0.003078,0.003438,0.003732,0.004100,0.004329,0.004687,0.004879
3,1,4,English,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.002435,0.002823,0.003117,0.003441,0.003788,0.004056,0.004285,0.004527,0.004843,0.005085
4,2,1,Arabic,0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.002465,0.002724,0.003079,0.003305,0.003425,0.003403,0.003450,0.003626,0.003977,0.004547


In [3]:
print("Test DataFrame:")
test_df.head()

Test DataFrame:


,writer,page_id,language,same_text,tortuosityHist10[0],tortuosityHist10[1],tortuosityHist10[2],tortuosityHist10[3],tortuosityHist10[4],tortuosityHist10[5],...,directions_hist1a2a3a4a5a6a7a8a9a10_220[210],directions_hist1a2a3a4a5a6a7a8a9a10_220[211],directions_hist1a2a3a4a5a6a7a8a9a10_220[212],directions_hist1a2a3a4a5a6a7a8a9a10_220[213],directions_hist1a2a3a4a5a6a7a8a9a10_220[214],directions_hist1a2a3a4a5a6a7a8a9a10_220[215],directions_hist1a2a3a4a5a6a7a8a9a10_220[216],directions_hist1a2a3a4a5a6a7a8a9a10_220[217],directions_hist1a2a3a4a5a6a7a8a9a10_220[218],directions_hist1a2a3a4a5a6a7a8a9a10_220[219]
0,283,1,Arabic,0,1.000000,0.000000,0.0,0.0,0.0,0.0,...,0.002271,0.002299,0.002386,0.002502,0.002622,0.002738,0.002817,0.002870,0.003048,0.003381
1,283,2,Arabic,1,0.994351,0.005649,0.0,0.0,0.0,0.0,...,0.002387,0.002451,0.002527,0.002613,0.002675,0.002746,0.002889,0.002945,0.003114,0.003291
2,283,3,English,0,1.000000,0.000000,0.0,0.0,0.0,0.0,...,0.002395,0.002636,0.002865,0.003035,0.003215,0.003402,0.003730,0.003870,0.004187,0.004385
3,283,4,English,1,1.000000,0.000000,0.0,0.0,0.0,0.0,...,0.002341,0.002563,0.002840,0.003072,0.003387,0.003650,0.003881,0.004138,0.004435,0.004678
4,284,1,Arabic,0,1.000000,0.000000,0.0,0.0,0.0,0.0,...,0.002135,0.002426,0.002851,0.003184,0.003554,0.003918,0.004146,0.004530,0.004865,0.005171
